## **Exercise**
Complete the following tasks:
1. Load the [dataset](https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset) from Kaggle.
2. Visualize the dataset and it's structure using appropriate libraries and plots.
3. Do some basic cleaning to handle missing values
4. Create the following features:
  *   Revenue
  *   DayOfWeek: to analyze sales trends by weekdays.
  *   TotalRevenue for each CustomerID
  *   Most popular product based on Revenue.
  *   Ordersize by summing Quantity for each InvoiceNo
5. Apply a lambda function to:
  * Segment customers into tiers based on TotalRevenue (e.g., "High", "Medium", "Low").
  * Extract key information from Description and add them as columns (e.g., presence of specific keywords like "Gift" or "Discount"). At least one extra column should be added
  * Categorize transactions as "Small", "Medium", or "Large" based on Revenue.
  * **Detect Seasonal Items:** Flag items as "Christmas"-themed if the description contains relevant words.
  * Classify customers as "Loyal", "Occasional", or "One-time" based on the number of purchases.
  *  **Identify Multi-Item Invoices:** Flag invoices with multiple unique items as "Multi-Item Order".
7. Wrap all fo the above into an ETL pipeline.

The hand-in exercise for this topic is Task 7 from the notebook “DE_Task.ipynb”. The
exercise asks you to wrap the functionality of other tasks in this notebook into an ETL
pipeline. Note that the GroupBY tasks (which are extra) are not needed in this ETL pipeline.
The data visualization parts are also not needed. The pipeline should contain some of the
data cleaning (such as removing duplicates and nulls) and then creation of features which
are part of task 4 and 5.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os

In [2]:
path = kagglehub.dataset_download("lakshmi25npathi/online-retail-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/marek/.cache/kagglehub/datasets/lakshmi25npathi/online-retail-dataset/versions/1


In [3]:
dataset_path = "/Users/marek/.cache/kagglehub/datasets/lakshmi25npathi/online-retail-dataset/versions/1"

files = os.listdir(dataset_path)
print(files)


['online_retail_II.xlsx']


In [4]:
file_path = "/Users/marek/.cache/kagglehub/datasets/lakshmi25npathi/online-retail-dataset/versions/1/online_retail_II.xlsx"

df = pd.read_excel(file_path)

In [5]:
def extract(dataset_path):

    file_path = dataset_path
    df = pd.read_excel(file_path)
    return df

def transform(df):

    df['Description'] = df.groupby('StockCode')['Description'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)

    df = df.dropna(subset=['Customer ID'])

    df.drop_duplicates(inplace=True)

    df.loc[:, 'Customer ID'] = df['Customer ID'].round().astype('int64')

    df.loc[:, 'Quantity'] = df['Quantity'].abs()

    df = df[~df['StockCode'].str.contains('TEST', case=False, na=False)]

    df.loc[:,'Revenue'] = df['Quantity'] * df['Price']
    df.loc[:,'DayOfWeek'] = df['InvoiceDate'].dt.day_of_week

    customer_revenue = df.groupby('Customer ID')[['Revenue', 'Quantity']].sum().reset_index().rename(columns={'Revenue': 'TotalRevenue'})
    df = pd.merge(df, customer_revenue[['Customer ID', 'TotalRevenue']], on='Customer ID', how='left')

    product_revenue = df.groupby('StockCode')['Revenue'].sum().reset_index().rename(columns={'Revenue': 'ProductRevenue'})
    df = pd.merge(df, product_revenue, on='StockCode', how='left')

    order_sizes = df.groupby('Invoice')['Quantity'].sum().reset_index().rename(columns={'Quantity': 'OrderSize'})
    df = pd.merge(df, order_sizes, on='Invoice', how='left')

    define_tier = lambda rev: 'Low' if rev < 1000 else ('Medium' if rev <= 10000 else 'High')
    df['Tier'] = df['TotalRevenue'].apply(define_tier)

    df['ProductType'] = df['Description'].apply(lambda x: 'Postage' if 'POSTAGE' in str(x).upper()
                                            else ('Discount' if 'DISCOUNT' in str(x).upper()
                                            else ('Manual' if 'MANUAL' in str(x).upper()
                                            else ('Adjustment' if 'ADJUSTMENT' in str(x).upper()
                                            else ('Gift' if 'GIFT' in str(x).upper()
                                            else ('Fee' if 'FEE' in str(x).upper()
                                            else ('Sample' if 'Sample' in str(x).upper()
                                            else 'Regular')))))))

    df['Color'] = df['Description'].apply(lambda x: 'Pink' if 'PINK' in str(x).upper()
                                        else ('Blue' if 'BLUE' in str(x).upper()
                                        else ('Red' if 'RED' in str(x).upper()
                                        else ('White' if 'WHITE' in str(x).upper()
                                        else ('Black' if 'BLACK' in str(x).upper()
                                        else ('Green' if 'GREEN' in str(x).upper()
                                        else ('Silver' if 'SILVER' in str(x).upper()
                                        else 'Other')))))))
    
    df['TransactionSize'] = df['Revenue'].apply(lambda x: 'Small' if x <= 50 else ('Medium' if x <= 500 else 'Large'))

    df['IsChristmas'] = df['Description'].apply(lambda x: True if 'CHRISTMAS' in str(x).upper() else False)

    invoices_per_customer = df.groupby('Customer ID')['Invoice'].nunique().reset_index().rename(columns={'Invoice' : 'Purchases'})
    df = pd.merge(df, invoices_per_customer, on='Customer ID', how='left')
    df['Engagement'] = df['Purchases'].apply(lambda x: 'One-Time' if x == 1 else ('Ocassional' if x <= 10 else 'Loyal'))

    items_per_invoice = df.groupby('Invoice')['StockCode'].nunique()
    df['IsMultiItem'] = df['Invoice'].map(lambda x: items_per_invoice[x] > 1)

    return df

def load(df, output_path):
    #df.to_csv(output_path, index=False)
    print("Data saved")
    return df


def ETL(dataset_path, output_path):
    df = extract(dataset_path)
    df_transformed = transform(df)
    df_orders_loaded = load(df_transformed, output_path)

    return df_orders_loaded



In [6]:
path = "/Users/marek/.cache/kagglehub/datasets/lakshmi25npathi/online-retail-dataset/versions/1/online_retail_II.xlsx"
output = "define a path"

df = ETL(path, output)
display(df.head())

/var/folders/yq/94jp9phd29s5g98ng2hwj0000000gn/T/ipykernel_91063/4278487719.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


Data saved


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,DayOfWeek,...,ProductRevenue,OrderSize,Tier,ProductType,Color,TransactionSize,IsChristmas,Purchases,Engagement,IsMultiItem
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,1,...,10093.45,166,Medium,Regular,Other,Medium,True,7,Ocassional,True
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,1,...,17138.45,166,Medium,Regular,Pink,Medium,False,7,Ocassional,True
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,1,...,23718.15,166,Medium,Regular,White,Medium,False,7,Ocassional,True
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,1,...,7348.50,166,Medium,Regular,Other,Medium,False,7,Ocassional,True
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,1,...,30355.59,166,Medium,Regular,Other,Small,False,7,Ocassional,True
